<a href="https://colab.research.google.com/github/askerzim/analise_consumidor/blob/main/analise_consumidor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Prevendo a Propensidade de Compra do Consumidor**

No exemplo abaixo, criei uma base aleatória com valores booleanos para simular clientes, afim de prever a propensão dos clientes potenciais. Usando essa propensão, com a área de negócios, decidiremos qual ponto de contato iremos fazer. (ex: fazer uma promoção de um determinado produto com 15% de descontos para esses clientes)

In [ ]:
import pandas as pd
import numpy as np


# Definindo o número de usuários
num_usuarios = 500

# Criando um DataFrame com os dados
df = pd.DataFrame({
    'id_usuario': range(1, num_usuarios + 1),
    'clique_imagem': np.random.randint(0, 2, num_usuarios),
    'viu_categoria': np.random.randint(0, 2, num_usuarios),
    'leu_detalhes': np.random.randint(0, 2, num_usuarios),
    'viu_video': np.random.randint(0, 2, num_usuarios),
    'add_lista': np.random.randint(0, 2, num_usuarios),
    'viu_similar': np.random.randint(0, 2, num_usuarios),
    'comprou': np.random.randint(0, 2, num_usuarios),
})

# Exibindo o DataFrame
df.head()


,id_usuario,clique_imagem,viu_categoria,leu_detalhes,viu_video,add_lista,viu_similar,comprou
0,1,0,0,1,1,1,0,1
1,2,0,0,1,1,0,1,1
2,3,1,0,1,1,0,1,1
3,4,1,0,0,0,0,1,0
4,5,1,1,1,1,1,1,0


 **OBS:** Colocarei uma prob maior nessas variáveis devido ao exemplo para ficar o mais próximo da realidade de compras online

In [ ]:
prob_comprou = 0.63  # Probabilidade de ser 1 se 'comprou' for 1
prob_nao_comprou = 0.37  # Probabilidade de ser 1 se 'comprou' for 0

for col in ['leu_review', 'comparou_prod', 'save_tarde', 'personalizou', 'add_lista']:
    df[col] = np.where(df['comprou'] == 1,
                       np.random.choice([0, 1], size=num_usuarios, p=[1-prob_comprou, prob_comprou]),
                       np.random.choice([0, 1], size=num_usuarios, p=[1-prob_nao_comprou, prob_nao_comprou]))


# **Fazendo uma análise de correlação**

In [ ]:
df.corr()['comprou']

id_usuario       0.058153
clique_imagem    0.064460
viu_categoria    0.014142
leu_detalhes    -0.008766
viu_video        0.032037
add_lista        0.236466
viu_similar     -0.037796
comprou          1.000000
leu_review       0.278928
comparou_prod    0.229158
save_tarde       0.168758
personalizou     0.289569
Name: comprou, dtype: float64

In [ ]:
# Eliminar colunas com baixa recomendação

preditores = df[['leu_review', 'comparou_prod', 'add_lista', 'save_tarde', 'personalizou']]
alvo = df.comprou

# **Divisão de Treinamento e Teste**

Dividir o modelo em dados de treino e teste na proporção 70:30
(70% para treinar e 30% para testar)

In [ ]:
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import sklearn.metrics

pred_train, pred_test, alvo_train, alvo_test = train_test_split(preditores, alvo, test_size=0.3)
print('Preditores - Treino: ', pred_train.shape, 'Preditores - Teste: ', pred_test.shape )

Preditores - Treino:  (350, 5) Preditores - Teste:  (150, 5)


# **Construir modelo e verificar precisão**

In [ ]:
from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()
classifier = classifier.fit(pred_train, alvo_train)

predicoes = classifier.predict(pred_test)

#Analisando a precisão das predições

sklearn.metrics.confusion_matrix(alvo_test, predicoes)

array([[45, 26],
       [26, 53]])

In [ ]:
#Precisão geral
sklearn.metrics.accuracy_score(alvo_test, predicoes)

0.6533333333333333

Ao invés de falar Sim ou Não para a previsão, farei um cálculo de probabilidade para mostrar a probabilidade do cliente potencial para comprar o produto ou não

In [ ]:
pred_prob = classifier.predict_proba(pred_test)
pred_prob[0,1]

0.7204882282769567

# **Previsões**

Com o modelo construído, usarei para prever os novos clientes aqui que estão acessando o site para fazer qualquer atividade.

Ex: Cliente Potencial acabou de acessar o site, usarei uma matriz de valores passadas (nessa mesma ordem) de "leu_review", "comparou_prod", "add_lista", "save_tarde", "personalizou". Sendo assim ela começara com 0

In [ ]:
# ["leu_review", "comparou_prod", "add_lista", "save_tarde", "personalizou"]
dados_cliente = np.array([1, 0, 0, 0, 1]).reshape(1, -1)
print(f'Usuário novo chegou ao site! \n Probabilidade de compra: {classifier.predict_proba(dados_cliente)[:,1]}')

Usuário novo chegou ao site! 
 Probabilidade de compra: [0.46077321]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


Portanto, a probabilidade de um usuário comprar quando ele entra no site (visita inicial) é de 7,67%. A cada mudança nas variaveis de maior correlação em relação a variavel 'comprou' (as que eu escolhi), essa probabilidade muda.